In [ ]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

In [ ]:
This is a temporary script file.
"""

In [ ]:
from collections import namedtuple

In [ ]:
from docplex.mp.model import Model

In [ ]:
import numpy as np

In [ ]:
C_i = 40
c_k = 6
    
def init():
    N = 240
    B = 30
    K = B * 3
    
    ell = []
    u = []
    phi_cap = []
    N_array = []
    phi_small = []
    
    B_array = []
    
    f_k = []
    K_array = []
    shelf_seg = []
    
    lmbd = []
    numerator = 0
    denominator = 0
    
    for j in range(1,N+1):
        temp = np.round(np.random.uniform(1,C_i/6.0))
        ell.append(temp)
        temp_2 = np.round(np.random.uniform(temp, 6.0))
        u.append(temp_2)
        temp_3 = np.round(np.random.uniform(1,25),2)
        phi_cap.append(temp_3)
        N_array.append(j)
        phi_small.append(0.1)
        
    for i in range(1,B+1):
        B_array.append(i)
        
    for k in range(1,K+1):
        f_k_instance = np.random.uniform(0,1)
        f_k.append(f_k_instance)
        shelf_seg.append(np.ceil(k/3.0))
        K_array.append(k)
        numerator = numerator + f_k_instance*c_k
        denominator = denominator + c_k
        if (k%3==0):
            lmbd.append(numerator/denominator)
            numerator = 0
            denominator = 0
        
    PRODUCTS = np.column_stack((N_array, ell, u, phi_cap, phi_small))
    SHELVES = np.column_stack(([B_array]))
    SEGMENTS = np.column_stack((K_array, shelf_seg,f_k))
    
    
    
    return PRODUCTS, SHELVES, SEGMENTS, lmbd

In [ ]:
class TProds(namedtuple("Prod", ["prod", "lower", "upper", "profit", "minreq"])):
    def __str__(self):
        return self.name
    
class TShelf(namedtuple("Shelf", ["shelf"])):
    def __str__(self):
        return self.name    
    
class TSegs(namedtuple("Seg", ["seg", "shelf", "attr"])):
    def __str__(self):
        return self.name    

In [ ]:
def build_problem(products, shelves, segments, rel, **kwargs):
    
    mdl = Model(name='APSA', **kwargs)
    
    mdl.products = [TProds(*prod_row) for prod_row in products]
    mdl.shelves = [TShelf(*shelf_row) for shelf_row in shelves]
    mdl.segments = [TSegs(*seg_row) for seg_row in segments]
    
    all_products = mdl.products
    all_shelves = mdl.shelves
    all_segments = mdl.segments
    # --- decision variables ---
    if (rel==True):
        mdl.x = mdl.continuous_var_matrix(all_shelves, all_products, 0, 1)
        mdl.y = mdl.continuous_var_matrix(all_segments, all_products, 0, 1)
        mdl.s = mdl.continuous_var_matrix(all_segments, all_products)
        mdl.z = mdl.continuous_var_matrix(all_products, all_products, 0, 1)
        mdl.q = mdl.continuous_var_matrix(all_segments, all_products, 0, 1)
    else:
        mdl.x = mdl.binary_var_matrix(all_shelves, all_products)
        mdl.y = mdl.binary_var_matrix(all_segments, all_products)
        mdl.s = mdl.continuous_var_matrix(all_segments, all_products)
        mdl.z = mdl.binary_var_matrix(all_products, all_products)
        mdl.q = mdl.binary_var_matrix(all_segments, all_products)
        
    # --- objective ---
    obj = mdl.sum(mdl.sum(j[3]* k[2] * mdl.s[k,j] / c_k for k in all_segments) for j in all_products)
    # --- constraints ---
    # constraint 1b
    for j in all_products:
        mdl.add_constraint(mdl.sum(mdl.x[i,j] for i in all_shelves) <= 1)
        
    # constraint 1c
    for k in all_segments:
        mdl.add_constraint(mdl.sum(mdl.s[k,j] for j in all_products)<= 6)
            
    # constraint 1d1
    for j in all_products:
        mdl.add_constraint(j.lower * mdl.sum(mdl.x[i,j] for i in all_shelves) <= mdl.sum(mdl.s[k,j] for k in all_segments))
    
    # constraint 1d2
    for j in all_products:
        mdl.add_constraint(mdl.sum(mdl.s[k,j] for k in all_segments) <= j.upper * mdl.sum(mdl.x[i,j] for i in all_shelves))
    
    # constraint 1e1
    for j in all_products:
        for k in all_segments:
            mdl.add_constraint(j.minreq * mdl.y[k,j] <= mdl.s[k,j])
            
    # constraint 1e2
    for j in all_products:
        for k in all_segments:
            mdl.add_constraint(mdl.s[k,j] <= min(c_k, j.upper) * mdl.y[k,j]) 
            
    # constraint 1f
    for j in all_products:
        for k1 in all_segments:
            for k2 in all_segments:
                for k3 in all_segments:
                    if k1.shelf == k2.shelf == k3.shelf and k1.seg < k2.seg < k3.seg:
                        mdl.add_constraint(mdl.s[k2,j] >= c_k * (mdl.y[k1,j] + mdl.y[k3,j] - 1))

    # constraint 1g
    for i in all_shelves:
        for j in all_products:
            gen = (k for k in all_segments if k.shelf == i.shelf)
            for k in gen:
                mdl.add_constraint(mdl.y[k,j] <= mdl.x[i,j])      
                
   # constraint 1h
    for i in all_shelves:
        for j in all_products:
            gen = (k for k in all_segments if k.shelf == i.shelf)
            mdl.add_constraint(mdl.x[i,j] <= mdl.sum(mdl.y[k,j] for k in gen))
            
    # constraint 1i
    for i in all_shelves:
        for j in all_products:
            gen1 = (k for k in all_segments if k.shelf == i.shelf and k.seg%3==0)
            gen2 = (k for k in all_segments if k.shelf == i.shelf and k.seg%3==1)
            gen_total = ((k1,k2) for k1 in gen1 for k2 in gen2 if k1.shelf==k2.shelf)
            for (k1,k2) in gen_total:
                mdl.add_constraint(mdl.q[k1,j] >= mdl.y[k1,j] + mdl.y[k2,j]-1)
    
    # constraint 1j
    for i in all_shelves:
        gen1 = (k for k in all_segments if k.shelf == i.shelf and k.seg%3!=2)
        for k in gen1:
            mdl.add_constraint(mdl.sum(mdl.q[k,j] for j in all_products)<=1)
    mdl.maximize(obj)
    
    return mdl

In [ ]:
def solve(model, **kwargs):
    model.parameters.mip.tolerances.mipgap = 0.0050
    model.parameters.timelimit = 3600
    sol = model.solve(log_output=True, **kwargs)
    return sol